In [1]:
import pandas as pd
import numpy as np

# Note
- 2020与2021年数据不再变更，不再进行处理，直接处理2022年全年数据，每月整体更新
- 2022年新增“企业简称”字段，可增加；企业简称按车型维度进行对照

### 处理步骤

0. 读入数据

In [26]:
df = pd.read_excel('2205_cn.xlsx',sheet_name=[0,1,2],header=3) # 一次性将所有sheet都读出
# raw20 = df[1]
# raw21 = df[0]
raw22 = df[0]

In [3]:
raw22.head()

,企业简称,品牌,国产进口,车辆类型,车型,HV/PHV/EV,１月,２月,３月,４月,５月,６月,７月,８月,９月,１０月,１１月,１２月,2022年
0,爱驰汽车,爱驰,国产,SUV,爱驰U5,EV,59.0,7.0,48.0,15.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,爱驰汽车,爱驰 Total,NaN,NaN,NaN,NaN,59.0,7.0,48.0,15.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,爱驰汽车 Total,NaN,NaN,NaN,NaN,NaN,59.0,7.0,48.0,15.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,阿斯顿马丁汽车,阿斯顿·马丁,进口,Cars,阿斯顿·马丁DB11,NaN,26.0,7.0,7.0,4.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,阿斯顿马丁汽车,阿斯顿·马丁,进口,Cars,阿斯顿·马丁DBS,NaN,2.0,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 1.5 提取企业-车型对照表

In [12]:
oem_model = raw22[['企业简称','品牌','车型','国产进口']].drop_duplicates().dropna(subset=['车型'])
# oem_model.groupby('车型').nunique().sort_values(by='企业简称',ascending=False)
# oem_model.head()
oem_model.to_csv('企业车型对照表.csv',encoding='gbk',index=False)

新的企业车型对照表是完整的，此字段2022年新增

- 2 将两个表内分别增加年份列后合并成为一个表

In [4]:
# raw20.rename(columns={'2020年':'year'},inplace=True)
# raw20.year = 2020
# raw21.rename(columns={'2021年':'year'},inplace=True)
# raw21.year = 2021
# raw = pd.concat([raw20,raw21],axis=0)

raw22.rename(columns={'2022年':'year'},inplace=True)
raw22.year = 2022

- 3 删除“品牌”列中含"Total"的行
- 4 将“燃料类型”列重命名，并将其中的NaN替换为ICE

In [5]:
raw = raw22.iloc[:,1:].dropna(subset=['品牌','国产进口'])
# raw = raw[-raw['品牌'].str.contains('Total')]

raw.rename(columns={'HV/PHV/EV':'fueltype'},inplace=True)
raw.fueltype.fillna('ICE',inplace=True)

- 5 表格变形
- 6 删除没有销量的行
- 7 去掉month中的“月”字，将字符串转换为数字
- 8 将车辆类型中的Cars替换为CAR

In [6]:
id_cols = raw.columns[[0,1,2,3,4,17]]
value_cols = raw.columns[5:17]
df = raw.melt(id_vars=id_cols,value_vars=value_cols,var_name='month',value_name='volume')

df.dropna(axis=0,subset=['volume'],inplace=True)
df.month = df.month.str.replace('月','')
df.车辆类型 = df.车辆类型.str.replace('Cars','CAR')

- 9 依次按年和月降序排列

In [7]:
df.sort_values(by=['year','month'],ascending=False,inplace=True)

In [8]:
df.head()

,品牌,国产进口,车辆类型,车型,fueltype,year,month,volume
4612,爱驰,国产,SUV,爱驰U5,EV,2022,５,27.0
4613,阿斯顿·马丁,进口,CAR,阿斯顿·马丁DB11,ICE,2022,５,12.0
4614,阿斯顿·马丁,进口,CAR,阿斯顿·马丁DBS,ICE,2022,５,3.0
4615,阿斯顿·马丁,进口,CAR,阿斯顿·马丁V8 Vantage,ICE,2022,５,12.0
4616,阿斯顿·马丁,进口,SUV,阿斯顿·马丁DBX,ICE,2022,５,34.0


- 10 导出最终的原始数据

In [9]:
df[df['month'].isin(['５'])].to_csv("2205_cn.csv",index=False,encoding='gbk')

In [10]:
df.month.unique()

array(['５', '４', '３', '２', '１'], dtype=object)